## chargemennt de la dataset spam.csv

In [116]:
import pandas as pd

spams = pd.read_csv("./spam.csv", header = 0, encoding = 'ISO-8859-14', sep = ',' )
spams

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


## pretraitement

suppression des colonnes vides et renommage des noms de colonnes 

In [117]:
spams = spams[['v1', 'v2']]
spams.columns = ['class', 'text']
spams

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


`codage one hot`

Vue pour utilisee la classification bayesienne, il est imperatif de transformer les textes de mail en des vecteurs numeriques leurs representant en s'appuyant sur l'ensemble des mots contenu dans ces derniers (appelé vocabulaire). 

Il existe plusieurs moyens de representation de texte, à l'instar du codage one hot dont le principe est : " de part un mot du vocabulaire, si dans le texte ce mot est utilise on mettra `1` dans la position du vecteur correspondant a ce mot et `0` sinon ".

pour le faire dans notre cas, nous avons choisi d'utiliser la classe `CountVectorizer` du module `sklearn.feature_extraction.text`, dont la particularité est qu'elle permet de conserver apres transformation des textes la matrice obtenu dans un objet de type `space matrix` permettant de mieux reduire la taille des donnees de la matrice en memoire lorsqu'il y'a grandement de valeur `0` inutile dans notre contexte 


In [118]:
from sklearn.feature_extraction.text import  CountVectorizer

# on instancie le converteur de donnees texte de sklearn
data_converter = CountVectorizer()

# on realise la conversion
spams_one_hot = data_converter.fit_transform(spams.text)

print(" dimension : ", spams_one_hot.shape)
print(" valeur de notre objet obtenu : ", spams_one_hot)
print(" matrice y contenu : \n", spams_one_hot.toarray())


 dimension :  (5572, 8681)
 valeur de notre objet obtenu :    (0, 3550)	1
  (0, 8029)	1
  (0, 4350)	1
  (0, 5920)	1
  (0, 2327)	1
  (0, 1303)	1
  (0, 5537)	1
  (0, 4087)	1
  (0, 1751)	1
  (0, 3634)	1
  (0, 8488)	1
  (0, 4476)	1
  (0, 1749)	1
  (0, 2048)	1
  (0, 7645)	1
  (0, 3594)	1
  (0, 1069)	1
  (0, 8266)	1
  (1, 5504)	1
  (1, 4512)	1
  (1, 4318)	1
  (1, 8391)	1
  (1, 5533)	1
  (2, 4087)	1
  (2, 3358)	1
  :	:
  (5570, 4218)	1
  (5570, 8312)	1
  (5570, 1084)	1
  (5570, 4615)	1
  (5570, 7039)	1
  (5570, 3308)	1
  (5570, 7627)	1
  (5570, 1438)	1
  (5570, 5334)	1
  (5570, 2592)	1
  (5570, 8064)	1
  (5570, 1778)	1
  (5570, 7049)	1
  (5570, 2892)	1
  (5570, 3470)	1
  (5570, 1786)	1
  (5570, 3687)	1
  (5570, 4161)	1
  (5570, 903)	1
  (5570, 1546)	1
  (5571, 7756)	1
  (5571, 5244)	1
  (5571, 4225)	2
  (5571, 7885)	1
  (5571, 6505)	1
 matrice y contenu : 
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


la nous avons bien une "sparse matrix" contenant une matrice de 5572 lignes (qui est le nombre d'observation) et de 8681 colonnes (qui est le nombre de mots du vocabulaire). nous pouvons aussi nous rendre compte que cette matrice contient enormement de 0

il faut noter qu'avoir un vocabulaire de 8681 mots est enorme et peut grandement impacter les performances de l'algorithme d'entrainement et donc il faudra minimiser le nombre de mots. 

Pour le faire nous pouvons:

        - mettre tous les mots en miniscules 
fait par defaut lors de la conversion avec CountVectorizer
                        
        - recuperer uniquement les racines des mots 
fait par defaut grace au l'expression reguliere par defaut `r"(?u)\b\w\w+\b"`)
    
        - supprimer les mots d'utilité courante (par exemple 'il' , 'nous', ... etc) qui ont une forte probablite d'utilisation (frequence)
egalement fait par defaut, cette frequence d'apparition est considere compris entre [0.7 .. 1]
    
        - donner un nombre de mots maximales
induira forcement une baisse de performance du model

Pour ce tutoriel nous definirons un nombre maximum de mots a 3000 (choix preferentiel)

In [119]:
# on instancie le converteur de donnees texte de sklearn
data_converter = CountVectorizer(max_features=3000)

# on realise la conversion
spams_one_hot = data_converter.fit_transform(spams.text)

print(" dimension : ", spams_one_hot.shape)


 dimension :  (5572, 3000)


## phase d'entrainement et evaluation

decoupage en partie de test et d'apprentissage homogene des donnes pretraitees

In [120]:
from sklearn.model_selection import train_test_split
data, labels = spams_one_hot.toarray(), spams['class'].values

X_train, X_test, y_train, y_test = train_test_split( data, labels, train_size=0.8, test_size=0.2, stratify= labels, random_state = 20)


entrainement

In [121]:
from sklearn.naive_bayes import  BernoulliNB

model = BernoulliNB()
model.fit(X_train, y_train)

BernoulliNB()

prediction et evaluation

In [122]:
from sklearn.metrics import confusion_matrix

predictions = model.predict(X_test)

print(y_test.shape)
print(predictions.shape)

def exactitude(m):
    result= 0
    for i in range(m.shape[0]):
        result += m[i,i]
    return result/(sum(sum(m)))
    
def precision(m):
    result=0
    for i in range(m.shape[0]):
        result += m[i,i]/sum(m[:,i])
    return result/2   
    
def rappel(m):
    result=0
    for i in range(m.shape[0]):
        result += m[i,i]/sum(m[i,:])
    return result/2 
def f_mesure(m):
    r=p=0
    for i in range(m.shape[0]):
        r += m[i,i]/sum(m[i,:])
        p += m[i,i]/sum(m[:,i])
    r/=2
    p/=2
    return 2*r*p/(r+p) 

matrice_confusion = confusion_matrix(y_true=y_test, y_pred=predictions)
print("matrice de confusion :\n" ,matrice_confusion)
print("exactitude :", exactitude(matrice_confusion))
print("precision :", precision(matrice_confusion))
print("rappel :", rappel(matrice_confusion))
print("f-mesure :", f_mesure(matrice_confusion))


(1115,)
(1115,)
matrice de confusion :
 [[964   2]
 [ 14 135]]
exactitude : 0.9856502242152466
precision : 0.9855432657143284
rappel : 0.9519849375408174
f-mesure : 0.968473483563818


malgres les valeurs de mettriques obtenu, il fautt noter que cette evaluation est basée sur les predictions faites sur les donnees de test pretraitées en meme temps que ceux de l'entrainnement.
Ainsi pour avoir la certitude que le modele est si bon, on pourrait reprétraiter les données et les utilises pour les tests 

In [123]:
from sklearn.metrics import confusion_matrix

# repretraitement des donnees
spams_one_hot = data_converter.transform(spams.text)
data, labels = spams_one_hot.toarray(), spams['class'].values

#predictions
predictions = model.predict(data)

print(labels.shape)
print(predictions.shape)

def exactitude(m):
    result= 0
    for i in range(m.shape[0]):
        result += m[i,i]
    return result/(sum(sum(m)))
    
def precision(m):
    result=0
    for i in range(m.shape[0]):
        result += m[i,i]/sum(m[:,i])
    return result/2   
    
def rappel(m):
    result=0
    for i in range(m.shape[0]):
        result += m[i,i]/sum(m[i,:])
    return result/2 
def f_mesure(m):
    r=p=0
    for i in range(m.shape[0]):
        r += m[i,i]/sum(m[i,:])
        p += m[i,i]/sum(m[:,i])
    r/=2
    p/=2
    return 2*r*p/(r+p) 

matrice_confusion = confusion_matrix(y_true=labels, y_pred=predictions)
print("matrice de confusion :\n" ,matrice_confusion)
print("exactitude :", exactitude(matrice_confusion))
print("precision :", precision(matrice_confusion))
print("rappel :", rappel(matrice_confusion))
print("f-mesure :", f_mesure(matrice_confusion))


(5572,)
(5572,)
matrice de confusion :
 [[4815   10]
 [  63  684]]
exactitude : 0.9868987796123474
precision : 0.9863378244733456
rappel : 0.956795055871153
f-mesure : 0.971341860786683
